In [4]:
import requests, json, xmltodict
import openai

def get_cid(bvid, max_page=10):
    """
    Get the cid of a bilibili video from its bvid.
    """

    url = "https://api.bilibili.com/x/web-interface/view?bvid=" + str(bvid)
    response = requests.get(url)
    res_dict = json.loads(response.text)

    cookie = r"LIVE_BUVID=AUTO3715893391262763; balh_mode=replace; balh_server_inner=__custom__; balh_is_closed=; buvid3=6117E460-4D82-4B15-BE1A-A55C9FD10FC618537infoc; buvid4=9518E84A-AA46-DBB3-2C8F-8030D361F84D25428-022050514-ly5N23H31hRIBxdxwOmqvQ==; buvid_fp_plain=undefined; hit-dyn-v2=1; b_nut=100; CURRENT_FNVAL=4048; _uuid=3C447151-E92D-BF57-10B97-E53BB85910C10252753infoc; rpdid=|(kkuum|~uRm0J'uY~|~mY|Ru; hit-new-style-dyn=1; SESSDATA=82dff401,1686423352,82d10*c2; bili_jct=36ea9e991cf030b5126e1d1558ac4e67; DedeUserID=14708458; DedeUserID__ckMd5=1bd4979b8de02688; i-wanna-go-back=-1; b_ut=5; nostalgia_conf=-1; header_theme_version=CLOSE; fingerprint=b2847ca1783dc8f8baadf44ec9d10a20; buvid_fp=9eb5471c66f7cf12232d2dc31ff4ce47; home_feed_column=4; PVID=2; CURRENT_QUALITY=0; bp_video_offset_14708458=769898593419001900; innersign=0; b_lsid=2CF10ED4A_186B6419CC3"

    title    = res_dict['data']['title']
    subtitle = res_dict['data']['subtitle']

    headers = {
        'Cookie': cookie,
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)

    subtitle_url = None
    if len(data['data']['subtitle']['list']) > 0:
        subtitle_url = data['data']['subtitle']['list'][0]['subtitle_url']


    num_comments = data['data']['stat']['reply']
    comments = []

    num_pages = num_comments // 20 + 1
    for n in range(1, min(num_pages, max_page) + 1):
        url = "https://api.bilibili.com/x/v2/reply?jsonp=jsonp&pn="+str(n)+"&type=1&oid="+str(bvid)+"&sort=1&nohot=1"
        req = requests.get(url)
        text = req.text
        json_text_list = json.loads(text)

        if json_text_list["data"]["replies"] is not None:
            for i in json_text_list["data"]["replies"]:
                comments.append("%s: %s"%(i["member"]["uname"],i["content"]["message"]))

    subtitle = ""
    if subtitle_url is not None:
        subtitle_response = requests.get(subtitle_url, headers=headers)
        subtitle = ",".join([json.loads(subtitle_response.text)['body'][i]['content'] for i in range(len(json.loads(subtitle_response.text)['body']))])

    return title, subtitle, comments

def get_prompt_summarize_subtitle(title, subtitle):
    """
    Get the prompt for the given cid.
    """

    prompt = "我希望你是一名专业的视频内容编辑，帮我总结视频的内容精华。请你将视频字幕文本进行总结，然后以无序列表的方式返回，不要超过5条。记得不要重复句子，确保所有的句子都足够精简，清晰完整，祝你好运！"
    prompt = "我希望你是一名专业的视频内容编辑，你的工作是基于视频的字幕将视频改写成一篇文章。请你将视频字幕文本进行总结，然后以无序列表的方式返回。记得不要重复句子，确保所有的句子都足够精简，清晰完整，祝你好运！"
    prompt += "\n视频标题：" + title
    prompt += "\n视频字幕：" + subtitle

    return prompt

def get_prompt_summarize_comments(title, comment):
    """
    Get the prompt for the given cid.
    """

    prompt  = "我希望你是一名专业的视频内容编辑，现在你需要阅读一条对视频的评论，判断这条评论的语气（严肃、戏谑等）、主要内容（个人经历、表达观点、提出建议等）以及评论的情感色彩（正面、负面、中性等）。"
    prompt += "以下是一个输出的例子：\n"
    prompt += "语气：严肃\n"
    prompt += "内容：表达观点\n"
    prompt += "情感：正面\n"
    prompt += "\n下面是一条评论，请你按照上面的格式进行输出：\n"
    prompt += comment


    return prompt

t ,s, cs = get_cid("BV1Px4y1T7FQ", max_page=1)


In [5]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user", 
    "content": get_prompt_summarize_subtitle(t, s)[:2000]
    },
  ],
  temperature = 0.5,
  top_p = 1,
  frequency_penalty = 0,
  presence_penalty = 0,
  max_tokens = 1000,
  stream = False, n = 1,
)
print(completion["choices"][0]["message"]["content"])



- 日本政府计划将国防预算增加 1/4，达到 5.34 万亿日元。
- 这是自卫队成立以来最大的一次军费增加。
- 军费增加的原因是为了加强日本的国防能力。
- 日本政府还计划购买新型战斗机和导弹防御系统等军事装备。
- 自卫队在过去的几年中也进行了一系列军备现代化改革。
- 这次军费增加引起了国内外的争议和关注。
- 一些人认为这是日本重返军国主义的表现，而另一些人则认为这是必要的国防措施。
- 自卫队也在近期进行了一次演习，模拟了钓鱼岛遭到外部威胁的情况。
- 这次演习也引起了中国的不满和反对。
- 自卫队还在回顾 1995 年的阪神大地震时期的救援行动，希望能够从中吸取经验教训。
- 总体来说，日本政府的军费增加和自卫队的军备现代化改革引起了国际社会的关注和反响。


In [6]:
import os
import openai
for ic, c in enumerate(cs):
  openai.api_key = os.getenv("OPENAI_API_KEY")
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
        "role": "user", 
      "content": get_prompt_summarize_comments(t, c)[:1000]
      },
    ],
    temperature = 0.5,
    top_p = 1,
    frequency_penalty = 0,
    presence_penalty = 0,
    max_tokens = 2000,
    stream = False, n = 1,
  )
  print("\n")
  print(completion["choices"][0]["message"]["content"])
  print(c)

  assert ic < 10






语气：严肃
内容：表达观点
情感：正面
从水下第一个生命萌芽: 。。。。。直到督公说到我的专业，我发现有些弹幕是真的为了喷而喷——对多数地区来说，卫星通讯就是性价比最高的方式




语气：严肃
内容：表达观点
情感：负面
小野塚小町: 看多了睡前消息，总会有种我国因为各种旧秩序把第五次工业变化拒之门外的感觉。。[无语]


语气：严肃
内容：表达观点、提出建议
情感：中性

评论分析：评论者在严肃地分析基站建设的成本和维护的复杂性，表达了自己的观点，认为基站建设和维护都需要耗费大量的时间和金钱。同时，评论者提出了建议，认为应该更加高效地利用现有的基站资源，避免浪费。整体情感是中性的，没有明显的正面或负面情感色彩。
希望启程: 基站不可能建更多了，运营商早没钱了，工程费继续打折，付款也越来越慢。建不起也用不起。
大多数乡村的家宽和4G信号覆盖都是赔钱的，没那么多人用。只是为了扶贫，但是占用运营费用维护费用可一点都不少。
基站建设就是一笔钱，建设周期也长，要找设计院做设计，找电力协调建变压器，开工了建设还要先修便道，然后平整土地建房建塔，最后才到放设备敷设光缆通信号。
日常使用得有人管理基站，看监控处理设备报警信息控制门禁登记进出人员。维护起来也很花钱，得请人维护上山的路，砍杂草砍树枝，定时进基站检查。打扫卫生，防火防虫防鼠，维护设备，检查发电机，更换蓄电池。一些没有配发电机的站得扛发电机给停电的站供电，来电了再回来扛走。线路维护还得请人




语气：严肃
内容：表达观点、提出建议
情感：正面

这条评论主要是在讨论星链技术的性能提升和业务增长。评论者认为星链2的技术可以对星链1进行魔改，提高性能。同时，评论者也提到了星链业务的爆发性增长，包括个人用户和企业用户的扩展，以及军方用户推出的星盾。评论者认为星链1已经无法满足需求，需要使用星链2的技术进行改进。最后，评论者还提到了使用氩作为推进剂的亮点。整个评论语气严肃，但情感色彩是正面的，表达了对星链技术的肯定，并提出了建议。
我们来吃你家大米啦: 星舰没稳定前 用星链2的技术对星链1作魔改

各个方面性能提高了2-4倍

星链业务去年一年爆发性增长

个人用户从25万发展到100万 而且年末一脚踢开了非洲大陆的大门

企业用户扩展了航空和邮轮 今年开始大规模部署

军方用户推出了星盾 同时在乌克兰大出风头



AssertionError: 